## Navigate to the Correct Directory

The following code navigates to the dataprocessing directory.

In [1]:
cd ../dataprocessing

/usr/local/google/home/carverforbes/activity-recognition/location/lbs/activity/audioset/dataprocessing


## Call the Import Statements

The following code imports the necessary code to run the code in the rest of this notebook.

In [2]:
# import statements
import audio_processing as ap

import functools
import os
import sys
import datetime
import gc

from absl import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import sklearn

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

print("Ran the import statements.")

Ran the import statements.


## Feature Extraction
Configure the following parameters to extract the desired features from a specified csv file to a specific destination directory.

In [3]:
# set logging to print logging.INFO logs
logging.set_verbosity(logging.INFO)

In [4]:
# arguments for audio_processing
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
filename = 'gunshot_50_50'
notebook = 'training_mfcc_dnn'
labels = ['Gunshot, gunfire']
available_features = ['chroma_stft',
                       'chroma_cqt',
                       'chroma_cens',
                       'melspectrogram',
                       'mfcc',
                       'rms',
                       'spectral_centroid',
                       'spectral_bandwidth',
                       'spectral_contrast',
                       'spectral_flatness',
                       'spectral_rolloff',
                       'poly_features',
                       'tonnetz',
                       'zero_crossing_rate']
features_to_extract = ['mfcc']
redo = False

In [5]:
def dataframe_to_csv(dataframe, dest_path):
    start_time = datetime.datetime.now()
    stop = dataframe.index.stop
    begin = 0
    end = 100
    count = 0
    while end < stop + 100:
        df = dataframe.iloc[begin:end, :].copy()
        for column in df.columns:
            for i in range(df[column].size):
                i = i + count * 100
                if isinstance(df[column][i], np.ndarray):
                    df[column][i] = df[column][i].tolist()
        if count == 0:
            df.to_csv(dest_path, index=False, header=True)
        else:
            df.to_csv(dest_path, mode='a', index=False, header=True)
        begin += 100
        end += 100
        count += 1
    end_time = datetime.datetime.now()
    function_duration = end_time - start_time
    
        
    print('Created the csv file at the destination path in {} seconds.'.format(
        function_duration))

In [6]:
def csv_to_dataframe(csv_path):
    df = pd.read_csv(csv_path)
    for column in df.columns:
        for i in range(df[column].size):
            if isinstance(df[column][i], str):
                temp = df[column][i][1:-1]
                temp = temp.replace(',', ' ').split(']')
                new_list = []
                for item in temp:
                    if item == '':
                        continue
                    item = item.replace('[', '').strip().split()
                    item = [float(num) for num in item]
                    arr = np.array(item)
                    new_list.append(arr)
                new_arr = np.array(new_list)
                df[column][i] = new_arr
    return df

In [ ]:
# Import the dataset as a pandas DataFrame object.
df = ap.output_df(src_dir, dest_dir, filename, labels, features_to_extract, redo)

In [8]:
# print the first 5 rows of the dataframe.
df.head()

,label,mfcc
0,1,"[[-303.27164, -310.85724, -324.51694, -325.635..."
1,1,"[[-282.00607, -289.7648, -299.44608, -298.7115..."
2,1,"[[-601.3228, -601.3228, -601.3228, -601.3228, ..."
3,1,"[[-244.3129, -188.5689, -145.10178, -155.65407..."
4,0,"[[-305.8128, -300.2478, -302.55612, -301.78333..."


## Data Preprocessing

Enter your data preprocessing here.

In [9]:
def fix_dataframe(dataframe):
    # temp bug fix for rows with None features
    # and rows with different number of elements per frame
    bad_rows = []
    for i in dataframe.index:
        if dataframe.mfcc[i] is None:
            bad_rows.append(i)
        elif dataframe.mfcc[i][0] is None:
            bad_rows.append(i)
        elif len(dataframe.mfcc[i]) != 20:
            bad_rows.append(i)
        elif len(dataframe.mfcc[i][0]) != 431:
            bad_rows.append(i)
    new_dataframe = dataframe.drop(bad_rows)
    return new_dataframe

In [10]:
def shuffle_dataframe(dataframe):
    # Shuffle the dataset/dataframe.
    dataframe = dataframe.reindex(np.random.permutation(dataframe.index))
    return dataframe

In [11]:
def flatten_features(dataframe):
    # Flatten melspectrogram's (128, 431) shaped features
    for i in dataframe.index:
        dataframe.mfcc[i] = dataframe.mfcc[i].flatten()
    return dataframe

In [12]:
def normalize_features(features):
    temp_x = []
    for arr in features:
        norm = np.linalg.norm(arr)
        arr = arr / norm
        temp_x.append(arr)
    features = np.array(temp_x, dtype=object)
    return features

In [13]:
def dataframe_to_x_y(dataframe):
    X = np.array(dataframe.mfcc.tolist(), dtype=object)
    y = np.array(dataframe.label.tolist())
    
    # Convert arrays of objects to arrays of floats.
    X = tf.keras.backend.cast_to_floatx(X)
    y = tf.keras.backend.cast_to_floatx(y)
    return X, y

In [19]:
def data_preprocessing(dataframe):
    print('Dropping incongruent features from dataframe')
    dataframe = fix_dataframe(dataframe)
    print('Flatten features in the dataframe')
    dataframe = flatten_features(dataframe)
    print('shuffling dataframe')
    dataframe = shuffle_dataframe(dataframe)
    print('Getting X, y out of dataframe')
    X, y = dataframe_to_x_y(dataframe)
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state = 42)
    return x_train, y_train, x_val, y_val

In [21]:
x_train, y_train, x_val, y_val = data_preprocessing(df)

Dropping incongruent features from dataframe
Flatten features in the dataframe


<ipython-input-11-f128fc5641ce>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.mfcc[i] = dataframe.mfcc[i].flatten()


shuffling dataframe
Getting X, y out of dataframe


## Building and Training Neural Network

In [22]:
def make_dir(path):
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
            return False
    return True

In [23]:
# Define the plotting function.
def plot_curve(epochs, hist, dest_path, notebook_filename, dataset_filename, list_of_metrics, list_of_hyperparameters):
    """Plot a curve of one or more classification metrics vs. epoch and save it to path."""  
    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)

    plt.legend()
    
    make_dir(dest_path)    
    path = os.path.join(dest_path, notebook_filename)
    make_dir(path)
    path = os.path.join(path, dataset_filename)
    make_dir(path)
        
    list_of_hyperparameters_temp = [str(item) for item in list_of_hyperparameters]
    filename = '_'.join(list_of_hyperparameters_temp)
    path = os.path.join(path, filename + '.png')
    plt.savefig(path, bbox_inches='tight')
    
    return plt


print("Defined the plot_curve function.")

Defined the plot_curve function.


## Training 1

In [24]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, optimizer, regularization, regularization_lambda):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    model = tf.keras.models.Sequential()

    # Add the input layer of 8620 nodes
    model.add(tf.keras.layers.Dense(units=8620, input_shape=(8620,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=8620, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.    
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)
    epochs = history.epoch
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

Defined the create_model and train_model functions.


In [ ]:
# The following variables are the hyperparameters.
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# A list of metrics to measure the performance of the model:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=classification_threshold),
    tf.keras.metrics.Precision(thresholds=classification_threshold, name='precision'),
    tf.keras.metrics.Recall(thresholds=classification_threshold, name="recall"),
]

# Create model
my_model = create_model(learning_rate, METRICS, optimizer=optimizer, regularization=regularization, regularization_lambda=regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, dest_dir, notebook, filename, list_of_metrics_to_plot, list_of_hyperparameters)
plot_curve(epochs, hist, dest_dir, notebook, filename, ['loss', 'val_loss'], list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8620)              74313020  
_________________________________________________________________
Hidden1 (Dense)              (None, 8620)              74313020  
_________________________________________________________________
Output (Dense)               (None, 1)                 8621      
Total params: 148,634,661
Trainable params: 148,634,661
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
351/351 [==============================] - 265s 756ms/step - loss: 2171.5479 - accuracy: 0.5059 - precision: 0.4655 - recall: 0.3735 - val_loss: 34.8981 - val_accuracy: 0.5527 - val_precision: 0.5130 - val_recall: 0.0600
Epoch 2/100
351/351 [==============================] - 264s 752ms/step - loss: 550.2348 - accuracy: 0.5130 - precision: 0.4789 - recall: 

## Training 2

In [ ]:
# The following variables are the hyperparameters.
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# A list of metrics to measure the performance of the model:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=classification_threshold),
    tf.keras.metrics.Precision(thresholds=classification_threshold, name='precision'),
    tf.keras.metrics.Recall(thresholds=classification_threshold, name="recall"),
]

# Create model
my_model = create_model(learning_rate, METRICS, optimizer=optimizer, regularization=regularization, regularization_lambda=regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, dest_dir, notebook, filename, list_of_metrics_to_plot, list_of_hyperparameters)
plot_curve(epochs, hist, dest_dir, notebook, filename, ['loss', 'val_loss'], list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Training 3

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, optimizer, regularization, regularization_lambda):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    model = tf.keras.models.Sequential()

    # Add the input layer of 8620 nodes
    model.add(tf.keras.layers.Dense(units=8620, input_shape=(8620,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=8620, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))
    # Include a dropout layer.
    model.add(tf.keras.layers.Dropout(0.2))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=8620, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.   
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)
    epochs = history.epoch
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

In [ ]:
# The following variables are the hyperparameters.
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# A list of metrics to measure the performance of the model:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=classification_threshold),
    tf.keras.metrics.Precision(thresholds=classification_threshold, name='precision'),
    tf.keras.metrics.Recall(thresholds=classification_threshold, name="recall"),
]

# Create model
my_model = create_model(learning_rate, METRICS, optimizer=optimizer, regularization=regularization, regularization_lambda=regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, dest_dir, notebook, filename, list_of_metrics_to_plot, list_of_hyperparameters)
plot_curve(epochs, hist, dest_dir, notebook, filename, ['loss', 'val_loss'], list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Training 4

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics, optimizer, regularization, regularization_lambda):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    model = tf.keras.models.Sequential()

    # Add the input layer of 8620 nodes
    model.add(tf.keras.layers.Dense(units=8620, input_shape=(8620,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=8620, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden1'))
    # Include a dropout layer.
    model.add(tf.keras.layers.Dropout(0.2))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=8620, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden2'))
    
    # Include a dropout layer.
    model.add(tf.keras.layers.Dropout(0.2))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=1000, 
                                  activation=activation,
                                  kernel_regularizer=regularization(regularization_lambda),
                                  name='Hidden3'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.   
    model.compile(optimizer=optimizer(lr=my_learning_rate),                                                   
                loss=loss,
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)
    epochs = history.epoch
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")

In [ ]:
# The following variables are the hyperparameters.
loss = tf.keras.losses.BinaryCrossentropy()
activation = 'relu'
optimizer = tf.keras.optimizers.RMSprop
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
regularization = tf.keras.regularizers.l2
regularization_lambda = 0.001
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           regularization_lambda,
                           label_name]


# A list of metrics to measure the performance of the model:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=classification_threshold),
    tf.keras.metrics.Precision(thresholds=classification_threshold, name='precision'),
    tf.keras.metrics.Recall(thresholds=classification_threshold, name="recall"),
]

# Create model
my_model = create_model(learning_rate, METRICS, optimizer=optimizer, regularization=regularization, regularization_lambda=regularization_lambda)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, dest_dir, notebook, filename, list_of_metrics_to_plot, list_of_hyperparameters)
plot_curve(epochs, hist, dest_dir, notebook, filename, ['loss', 'val_loss'], list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])